# <font color='red'>ch10_DB연동</font>

##  SQLite 데이터 베이스 연결

###  SQLite와 파이썬
- SQLite 데이터 베이스는 별도의 서버프로세스 없이 SQL을 사용하여 DB액세스할 수 있도록 만든 간단한 디스크 기반 데이터베이스를 제공하는 C라이브러리
- SQLite는 프로토타입을 만들 때까지 사용. 정식 제품은 오라클이나 다른 DBMS를 이식
- DB browser for SQLite (https://sqlitebrowser.org/dl)

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

### 데이터 베이스 연결

In [6]:
# DB 연결 객체 생성시, 파일이 없으면 파일 생성. 파일이 존재하면, 해당 파일과 연동
conn = sqlite3.connect('data/ch10_example.db')
conn

In [7]:
# cursor 객체 반환. cursor 는 SQL문을 실행시키고 결과데이터를 조회하는데 사용
cursor = conn.cursor()

In [16]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [15]:
cursor.execute("DROP TABLE MEMBER")

In [17]:
cursor.execute("INSERT INTO MEMBER VALUES('홍길동',20,'H@HONG.com')")
cursor.execute("INSERT INTO MEMBER VALUES('이길동',23,'y@HONG.com')")
cursor.execute("INSERT INTO MEMBER VALUES('김길동',30,'k@HONG.com')")

In [19]:
conn.commit() # 커밋되기전 파일 취소 rollback()

In [23]:
cursor.execute('SELECT name FROM MEMBER') # 시행 결과는 cursor가 가짐

In [24]:
# select 문의 결과를 받는 함수 :
# fetchall() - 결과를 모두 받을 때
# fetchone() - 결과를 한 행씩 받을 때
# fetchmany(n) - 결과를 n행 받을 때 
cursor.fetchall()
# 결과는 리스트안에 튜플로 출력

[('홍길동',), ('이길동',), ('김길동',)]

In [25]:
# 한번 패치한 파일을 여러번 사용하려면 다시 불러오거나 변수에 담아서 써야한다.
cursor.fetchall()

[]

In [26]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()

In [29]:
for member in members:
    print(member)

('홍길동', 20, 'H@HONG.com')
('이길동', 23, 'y@HONG.com')
('김길동', 30, 'k@HONG.com')


In [35]:
# fetchone() - 데이터 하나씩 불러옴. 더 불러올 데이터가 없으면 None
cursor.execute("SELECT * FROM MEMBER")
while True:
    member = cursor.fetchone()
    if member is None:
        break;
    print(member)

('홍길동', 20, 'H@HONG.com')
('이길동', 23, 'y@HONG.com')
('김길동', 30, 'k@HONG.com')


In [37]:
cursor.execute("SELECT * FROM MEMBER")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'H@HONG.com')
('이길동', 23, 'y@HONG.com')


In [38]:
cursor.close() # 생략가능
conn.close() # 생략불가

### SQL 구문에서 파라미터 사용하기
- qmark, named(추천) 방법

In [41]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE EMAIL IN('k@HONG.com','H@HONG.com')")
print(cursor.fetchall())

[('홍길동', 20, 'H@HONG.com'), ('김길동', 30, 'k@HONG.com')]


In [43]:
# qmark 파라미터 사용법
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?,?)",('홍길동','김철수'))
print(cursor.fetchall())

[('홍길동', 20, 'H@HONG.com'), ('김철수', 40, 'K@HONG.COM')]


In [46]:
name1 = input('검색할 이름1 : ')
name2 = input('검색할 이름2 : ')
names = (name1,name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?,?)",names)
print(cursor.fetchall())

검색할 이름1 : 홍길동
검색할 이름2 : 김철수
[('홍길동', 20, 'H@HONG.com'), ('김철수', 40, 'K@HONG.COM')]


In [53]:
# named 파라미터 사용법
name = input('검색하고자 하는 이름 : ')
cursor.execute("SELECT * FROM MEMBER WHERE NAME = :name",{'name':name})
members = cursor.fetchall()
if len(members)==0:
    print('해당 이름의 데이터가 없습니다')
else:
    print(members)

검색하고자 하는 이름 : 홍길동
[('홍길동', 20, 'H@HONG.com')]


In [54]:
name = input('입력할 이름 ?')
try:
    age = int(input('입력할 이름 ?'))
except:
    print('숫자를 입력해라')
    age=20
email = input('입력할 메일 ?')
cursor.execute("INSERT INTO MEMBER VALUES (:name,:age,:email)",{'name':name,'age':age,'email':email})

입력할 이름 ?어길동
입력할 이름 ?33
입력할 메일 ?eh@hong.com


In [58]:
cursor.execute("SELECT * FROM MEMBER")
cursor.fetchall()
conn.commit()

In [59]:
cursor.close()
conn.close()

## 오라클 데이터 베이스 연결

In [61]:
# 데이터 베이스 설정 (DBMS 서버 주소, 포트번호, sid등을 설정)
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521,sid='xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [62]:
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)
conn
# 에러 발생시 VC_redist.x64.exe 인스톨 해볼 것 !

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [ ]:
# conn 얻어오는 방법 (1)
oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521,sid='xe')
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)

In [64]:
conn.close()

In [65]:
# conn 얻어오는 방법 (2)
# dsn 직접 입력
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [66]:
#  cursor 얻어오기
cursor = conn.cursor()

In [67]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [68]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3300.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5500.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [69]:
sql = "SELECT * FROM EMP WHERE DEPTNO= :deptno"
deptno = input('검색하고자 하는 부서 번호는 ?')
cursor.execute(sql,{'deptno':deptno})
for emp in cursor:
    print(emp)

검색하고자 하는 부서 번호는 ?30
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7900, 'JAMES', 'CLERK', 7698, datetime.datetime(1981, 12, 3, 0, 0), 950.0, None, 30)


In [71]:
import pandas as pd
cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() # 튜플 리스트
data_df  = pd.DataFrame(data) # 데이터 프레임으로 변환
data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3300.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5500.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [76]:
# Header 변경방법
cursor.description # 각 필드 특징 정보 (필드명,type,display_size,내부크기,정확도,scale,nullable)

SyntaxError: invalid syntax (<ipython-input-76-910ad590236f>, line 2)

In [81]:
col=[row[0] for row in cursor.description]

In [82]:
data_df.columns = col

In [83]:
data_df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3300.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5500.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [84]:
cursor.close()
conn.close()

## MariaDB 데이터베이스 연결(MySQL 동일)

In [86]:
import pymysql

In [88]:
conn = pymysql.connect(host="localhost",port=3306,db="kimdb",user="root",passwd="mysql",charset="utf8",autocommit=True)
conn

In [90]:
cursor = conn.cursor()
sql = "select * from personal"
cursor.execute(sql)
result = cursor.fetchall()
for r in result:
    print(r)

(1001, 'bill', 'president', None, datetime.date(1989, 1, 10), 7000, None, 10)
(1111, 'smith', 'manager', 1001, datetime.date(1990, 12, 17), 1000, None, 10)
(1112, 'ally', 'salesman', 1116, datetime.date(1991, 2, 20), 1600, 500, 30)
(1113, 'word', 'salesman', 1116, datetime.date(1992, 2, 24), 1450, 300, 30)
(1114, 'james', 'manager', 1001, datetime.date(1990, 4, 12), 3975, None, 20)
(1116, 'johnson', 'manager', 1001, datetime.date(1991, 5, 1), 3550, None, 30)
(1118, 'martin', 'analyst', 1111, datetime.date(1991, 9, 9), 3450, None, 10)
(1121, 'kim', 'clerk', 1114, datetime.date(1990, 12, 8), 4000, None, 20)
(1123, 'lee', 'salesman', 1116, datetime.date(1991, 9, 23), 1200, 0, 30)
(1226, 'park', 'analyst', 1111, datetime.date(1990, 1, 3), 2500, None, 10)


In [94]:
personal = pd.DataFrame(result)
personal.columns = [row[0] for row in cursor.description]
personal

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20
5,1116,johnson,manager,1001.0,1991-05-01,3550,NaN,30
6,1118,martin,analyst,1111.0,1991-09-09,3450,NaN,10
7,1121,kim,clerk,1114.0,1990-12-08,4000,NaN,20
8,1123,lee,salesman,1116.0,1991-09-23,1200,0.0,30
9,1226,park,analyst,1111.0,1990-01-03,2500,NaN,10


In [95]:
cursor.close()
conn.close()